# 🚀 Welcome to Your ADK Adventure! 🚀

Welcome, Agent Architect! In this notebook, we're going on a journey to build and command our own fleet of AI agents using the Google Agent Development Kit (ADK).

We'll cover three awesome levels of agent creation:
1.  **Your First Agent (The Day Trip Genie 🧞):** Learn the absolute basics of creating and running a single, powerful agent.
2.  **Agent with a Memory (The Progressive Planner 🗺️):** See how agents can remember your conversation within a 'session' to handle multi-step tasks.
3.  **Multi-Agent System (The Agent Router 🚦):** Assemble a team of specialized agents and build a smart router to delegate tasks to the right expert.

-------------
### ⚠️ Important Prerequisite: Setup Your Environment! ⚠️
-----------------------------------------------------------------------------
 Before you run this file, please make sure you have completed the initial
setup steps from the Google Codelabs instructions.

This includes:

1. Setting up a Google Cloud project with billing.
2. Getting your free Gemini API Key.
3. (optional) Run the ADK Web.

👉 Follow the official guide here:
    https://codelabs.developers.google.com/onramp/instructions#0

Once you've done that, you'll be all set to run this demo!
 -----------------------------------------------------------------------------


Let's get this adventure started!

```
 /\_/\     /\_/\     /\_/\      /\_/\       /\_/\
( ^_^ )   ( -.- )   ( >_< )   ( =^.^= )    ( o_o )
 > 🌸 <     > z <    > 😾 <      > ♥ <      > 😎 <
```


## Part 0: Setup & Authentication 🔑

First things first, let's get all our tools ready. This step installs the necessary libraries and securely configures your Google API key so your agents can access the power of Gemini.

In [ ]:
!pip install google-adk google-generativeai google-cloud-aiplatform -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.agents import Agent, LlmAgent, BaseAgent
from google.adk.agents import Agent, SequentialAgent
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from google.colab import auth
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [ ]:
auth.authenticate_user()

Enter your Google Cloud Project ID: neon-emitter-458622-e3


In [ ]:
# fmt: on
PROJECT_ID = "placeholder" # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION = "us-central1"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"

In [ ]:
# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

```
+-----------------------------------------------------+
|         Adaptive Multi-Day Trip Agent 🗺️           |
|-----------------------------------------------------|
|  Model: gemini-2.5-flash                            |
|  Description:                                       |
|   Builds multi-day travel itineraries step-by-step, |
|   remembers previous days, adapts to feedback       |
|-----------------------------------------------------|
|  🔧 Tools:                                          |
|   - Google Search                                   |
|-----------------------------------------------------|
|  🧠 Capabilities:                                   |
|   - Memory of past conversation & preferences       |
|   - Progressive planning (1 day at a time)          |
|   - Adapts to user feedback                         |
|   - Ensures activity variety across days            |
+-----------------------------------------------------+

            ▲
            |
    +---------------------------+
    |     User Interaction      |
    |---------------------------|
    | - Destination             |
    | - Trip duration           |
    | - Interests & feedback    |
    +---------------------------+

            |
            ▼

+-----------------------------------------------------+
|        Day-by-Day Itinerary Generation              |
|-----------------------------------------------------|
|  🗓️ Day N Output (Markdown format):                 |
|   - Morning / Afternoon / Evening activities        |
|   - Personalized & context-aware                    |
|   - Changes accepted, feedback acknowledged         |
+-----------------------------------------------------+

            |
            ▼

+-----------------------------------------------------+
|        Next Day Planning Triggered 🚀               |
|-----------------------------------------------------|
| - Builds on prior days                              |
| - Avoids repetition                                 |
| - Asks user for confirmation before proceeding      |
+-----------------------------------------------------+
```


In [ ]:
# --- Agent Definition: The Adaptive Planner ---

def create_multi_day_trip_agent():
    """Create the Progressive Multi-Day Trip Planner agent"""
    return Agent(
        name="multi_day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent that progressively plans a multi-day trip, remembering previous days and adapting to user feedback.",
        instruction="""
        You are the "Adaptive Trip Planner" 🗺️ - an AI assistant that builds multi-day travel itineraries step-by-step.

        Your Defining Feature:
        You have short-term memory. You MUST refer back to our conversation to understand the trip's context, what has already been planned, and the user's preferences. If the user asks for a change, you must adapt the plan while keeping the unchanged parts consistent.

        Your Mission:
        1.  **Initiate**: Start by asking for the destination, trip duration, and interests.
        2.  **Plan Progressively**: Plan ONLY ONE DAY at a time. After presenting a plan, ask for confirmation.
        3.  **Handle Feedback**: If a user dislikes a suggestion (e.g., "I don't like museums"), acknowledge their feedback, and provide a *new, alternative* suggestion for that time slot that still fits the overall theme.
        4.  **Maintain Context**: For each new day, ensure the activities are unique and build logically on the previous days. Do not suggest the same things repeatedly.
        5.  **Final Output**: Return each day's itinerary in MARKDOWN format.
        """,
        tools=[google_search]
    )

multi_day_agent = create_multi_day_trip_agent()
print(f"🗺️ Agent '{multi_day_agent.name}' is created and ready to plan and adapt!")

🗺️ Agent 'multi_day_trip_agent' is created and ready to plan and adapt!


### Scenario 1a: Agent WITH Memory (Using a SINGLE Session) ✅

First, let's see the correct way to do it. We will use the **exact same `trip_session` object** for the entire conversation. Watch how the agent remembers the context from Turn 1 to correctly handle the requests in Turn 2 and 3.

In [ ]:
# --- Scenario 2: Testing Adaptation and Memory ---

async def run_adaptive_memory_demonstration():
    print("### 🧠 DEMO 2: AGENT THAT ADAPTS (SAME SESSION) ###")

    # Create ONE session that we will reuse for the whole conversation
    trip_session = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"Created a single session for our trip: {trip_session.id}")

    # --- Turn 1: The user initiates the trip ---
    query1 = "Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food."
    print(f"\n🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(multi_day_agent, query1, trip_session, my_user_id)

    # --- Turn 2: The user gives FEEDBACK and asks for a CHANGE ---
    # We use the EXACT SAME `trip_session` object!
    query2 = "That sounds pretty good, but I'm not a huge fan of castles. Can you replace the morning activity for Day 1 with something else historical?"
    print(f"\n🗣️ User (Turn 2 - Feedback): '{query2}'")
    await run_agent_query(multi_day_agent, query2, trip_session, my_user_id)

    # --- Turn 3: The user confirms and asks to continue ---
    query3 = "Yes, the new plan for Day 1 is perfect! Please plan Day 2 now, keeping the food theme in mind."
    print(f"\n🗣️ User (Turn 3 - Confirmation): '{query3}'")
    await run_agent_query(multi_day_agent, query3, trip_session, my_user_id)

await run_adaptive_memory_demonstration()

### 🧠 DEMO 2: AGENT THAT ADAPTS (SAME SESSION) ###
Created a single session for our trip: 0518d694-a0c2-432f-8d56-e7d78fa6b116

🗣️ User (Turn 1): 'Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '0518d694-a0c2-432f-8d56-e7d78fa6b116'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Hello! I'd love to help you plan your 2-day trip to Lisbon, focusing on historic sites and delicious local food.

Let's start with **Day 1**. How about this itinerary?

**Day 1: Historic Alfama & Baixa Flavors**

*   **Morning (9:00 AM - 1:00 PM): Explore Alfama District.** Begin your day by wandering through the narrow, winding streets of Alfama, Lisbon's oldest district. Visit the **Lisbon Cathedral (Sé de Lisboa)**, a striking example of Romanesque architecture, and then head up to **São Jorge Castle (Castelo de São Jorge)** for panoramic views of the city and the Tagus R

Hello! I'd love to help you plan your 2-day trip to Lisbon, focusing on historic sites and delicious local food.

Let's start with **Day 1**. How about this itinerary?

**Day 1: Historic Alfama & Baixa Flavors**

*   **Morning (9:00 AM - 1:00 PM): Explore Alfama District.** Begin your day by wandering through the narrow, winding streets of Alfama, Lisbon's oldest district. Visit the **Lisbon Cathedral (Sé de Lisboa)**, a striking example of Romanesque architecture, and then head up to **São Jorge Castle (Castelo de São Jorge)** for panoramic views of the city and the Tagus River.
*   **Lunch (1:00 PM - 2:30 PM): Traditional Portuguese Cuisine in Alfama.** Enjoy a traditional Portuguese lunch at a local tasca in Alfama. Look for dishes like *Bacalhau à Brás* (codfish with scrambled eggs and potatoes) or *Sardinhas Assadas* (grilled sardines, especially if you're there in summer).
*   **Afternoon (2:30 PM - 6:00 PM): Baixa and Rossio Square.** Descend from Alfama into the elegant, grid-patterned streets of the **Baixa district**, rebuilt after the 1755 earthquake. Stroll through **Praça do Comércio**, one of Europe's largest squares, and then make your way to **Rossio Square (Praça do Rossio)**, a lively central hub with beautiful fountains and historic buildings.
*   **Evening (7:00 PM onwards): Dinner and Fado in Chiado.** Head to the sophisticated **Chiado district** for dinner. Afterwards, experience an authentic **Fado show**, Portugal's soulful music, often found in intimate settings in Alfama or Chiado.

How does this sound for your first day in Lisbon?

--------------------------------------------------


🗣️ User (Turn 2 - Feedback): 'That sounds pretty good, but I'm not a huge fan of castles. Can you replace the morning activity for Day 1 with something else historical?'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '0518d694-a0c2-432f-8d56-e7d78fa6b116'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Okay, I understand! No problem at all, we can certainly swap out the castle for another fantastic historical site.

Here’s a revised plan for your **Day 1**, focusing on historic Alfama and Baixa flavors, with a different morning activity:

**Day 1: Historic Alfama & Baixa Flavors**

*   **Morning (9:00 AM - 1:00 PM): Explore Alfama District and Lisbon Cathedral.** Begin your day by wandering through the narrow, winding streets of Alfama, Lisbon's oldest district. Visit the impressive **Lisbon Cathedral (Sé de Lisboa)**, a striking example of Romanesque architecture. Afterwards, explore the **National Panth

Okay, I understand! No problem at all, we can certainly swap out the castle for another fantastic historical site.

Here’s a revised plan for your **Day 1**, focusing on historic Alfama and Baixa flavors, with a different morning activity:

**Day 1: Historic Alfama & Baixa Flavors**

*   **Morning (9:00 AM - 1:00 PM): Explore Alfama District and Lisbon Cathedral.** Begin your day by wandering through the narrow, winding streets of Alfama, Lisbon's oldest district. Visit the impressive **Lisbon Cathedral (Sé de Lisboa)**, a striking example of Romanesque architecture. Afterwards, explore the **National Pantheon (Panteão Nacional)**, originally a church, which now serves as the final resting place for important Portuguese figures. Its magnificent dome offers great views of the city.
*   **Lunch (1:00 PM - 2:30 PM): Traditional Portuguese Cuisine in Alfama.** Enjoy a traditional Portuguese lunch at a local tasca in Alfama. Look for dishes like *Bacalhau à Brás* (codfish with scrambled eggs and potatoes) or *Sardinhas Assadas* (grilled sardines, especially if you're there in summer).
*   **Afternoon (2:30 PM - 6:00 PM): Baixa and Rossio Square.** Descend from Alfama into the elegant, grid-patterned streets of the **Baixa district**, rebuilt after the 1755 earthquake. Stroll through **Praça do Comércio**, one of Europe's largest squares, and then make your way to **Rossio Square (Praça do Rossio)**, a lively central hub with beautiful fountains and historic buildings.
*   **Evening (7:00 PM onwards): Dinner and Fado in Chiado.** Head to the sophisticated **Chiado district** for dinner. Afterwards, experience an authentic **Fado show**, Portugal's soulful music, often found in intimate settings in Alfama or Chiado.

How does this revised Day 1 morning activity sound to you?

--------------------------------------------------


🗣️ User (Turn 3 - Confirmation): 'Yes, the new plan for Day 1 is perfect! Please plan Day 2 now, keeping the food theme in mind.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '0518d694-a0c2-432f-8d56-e7d78fa6b116'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Excellent! I'm glad Day 1 is perfect. Let's move on to **Day 2**, where we'll explore another historic area and continue our culinary journey through Lisbon.

Here's a proposal for your second day:

**Day 2: Maritime History & Culinary Delights of Belém**

*   **Morning (9:30 AM - 1:00 PM): Discover Belém's Maritime Heritage.** Start your day by taking a tram or taxi to the historic Belém district. Begin with a visit to the magnificent **Jerónimos Monastery (Mosteiro dos Jerónimos)**, a UNESCO World Heritage site and a masterpiece of Manueline architecture, closely associated with Portugal's Age of Discoveries. Afterwards, walk along the Tagus Ri

Excellent! I'm glad Day 1 is perfect. Let's move on to **Day 2**, where we'll explore another historic area and continue our culinary journey through Lisbon.

Here's a proposal for your second day:

**Day 2: Maritime History & Culinary Delights of Belém**

*   **Morning (9:30 AM - 1:00 PM): Discover Belém's Maritime Heritage.** Start your day by taking a tram or taxi to the historic Belém district. Begin with a visit to the magnificent **Jerónimos Monastery (Mosteiro dos Jerónimos)**, a UNESCO World Heritage site and a masterpiece of Manueline architecture, closely associated with Portugal's Age of Discoveries. Afterwards, walk along the Tagus River to see the iconic **Belém Tower (Torre de Belém)** and the **Monument to the Discoveries (Padrão dos Descobrimentos)**, both symbols of Portugal's rich maritime history.
*   **Lunch (1:00 PM - 2:00 PM): Original Pastéis de Nata.** No trip to Belém is complete without tasting the warm, custard tarts from **Pastéis de Belém**. Enjoy them fresh from the oven, perhaps with a sprinkle of cinnamon, for a truly authentic Lisbon food experience.
*   **Afternoon (2:00 PM - 5:30 PM): Lisbon's Artistic Side & Panoramic Views.** Head back towards central Lisbon. You could explore the vibrant street art and trendy shops of **LX Factory**, or for more historical views, take a ride on the iconic **Santa Justa Lift** for panoramic city views. Alternatively, wander through the **Bairro Alto** district during the day to see its charming streets before they come alive at night, perhaps stopping at a *miradouro* (viewpoint) like Miradouro de São Pedro de Alcântara for stunning vistas.
*   **Evening (6:30 PM onwards): Diverse Flavors at Time Out Market.** For your final dinner, immerse yourself in the culinary buzz of the **Time Out Market (Mercado da Ribeira)**. This historic market hall has been transformed into a vibrant food hall, featuring stalls from some of Lisbon's best chefs and restaurants, offering a wide array of local and international dishes. It’s a perfect spot to sample a variety of foods in a lively atmosphere.

How does this sound for your second day in Lisbon, blending more history with some fantastic food experiences?

--------------------------------------------------



### Scenario 1b: Agent WITHOUT Memory (Using SEPARATE Sessions) ❌

Now, let's see what happens if we mess up our session management. Here, we'll give the agent a case of amnesia by creating a **brand new, separate session for each turn**.

Pay close attention to the agent's response to the second query. Because it's in a new session, it has no memory of the trip to Lisbon we just discussed!

In [ ]:
# --- Scenario 1b: Demonstrating Memory FAILURE ---

async def run_memory_failure_demonstration():
    print("\n" + "#"*60)
    print("### 🧠 DEMO 2b: AGENT WITH AMNESIA (SEPARATE SESSIONS) ###")
    print("#"*60)

    # --- Turn 1: The user initiates the trip in the FIRST session ---
    query1 = "Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food."
    session_one = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"\nCreated a session for Turn 1: {session_one.id}")
    print(f"🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(multi_day_agent, query1, session_one, my_user_id)

    # --- Turn 2: The user asks to continue... but in a completely NEW session ---
    query2 = "Yes, that looks perfect! Please plan Day 2."
    session_two = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"\nCreated a BRAND NEW session for Turn 2: {session_two.id}")
    print(f"🗣️ User (Turn 2): '{query2}'")
    await run_agent_query(multi_day_agent, query2, session_two, my_user_id)

await run_memory_failure_demonstration()


############################################################
### 🧠 DEMO 2b: AGENT WITH AMNESIA (SEPARATE SESSIONS) ###
############################################################

Created a session for Turn 1: 1964b216-5412-4df6-a727-773af9cee36e
🗣️ User (Turn 1): 'Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '1964b216-5412-4df6-a727-773af9cee36e'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Hello! A 2-day trip to Lisbon focused on historic sites and local food sounds fantastic. I can definitely help you plan that. I'll break it down day by day.

Here's a proposed itinerary for your first day in Lisbon:

**Day 1: Alfama's Charms and Castle Views**

*   **Morning (9:00 AM - 1:00 PM): Explore the Alfama District & Lisbon Cathedral.**
    Start your day by immersing yourself in Alfama, Lisbon's oldest neighborhood. This historic district is chara

Hello! A 2-day trip to Lisbon focused on historic sites and local food sounds fantastic. I can definitely help you plan that. I'll break it down day by day.

Here's a proposed itinerary for your first day in Lisbon:

**Day 1: Alfama's Charms and Castle Views**

*   **Morning (9:00 AM - 1:00 PM): Explore the Alfama District & Lisbon Cathedral.**
    Start your day by immersing yourself in Alfama, Lisbon's oldest neighborhood. This historic district is characterized by its labyrinthine alleys, hidden squares, and traditional Fado houses, reflecting a pre-earthquake past. Take your time wandering through the narrow, cobbled streets. Within Alfama, you'll find the **Lisbon Cathedral (Sé de Lisboa)**, the oldest and most important church in the city, with a history dating back to 1147. Its Romanesque style and fortress-like appearance make it a notable historic site. You can also visit its Gothic cloisters and treasury.

*   **Lunch (1:00 PM - 2:30 PM): Traditional Portuguese Lunch in Alfama.**
    Enjoy a local meal in one of Alfama's charming restaurants. The area offers traditional Portuguese food options. You might even find some places with outdoor seating areas.

*   **Afternoon (2:30 PM - 5:30 PM): Discover St. George's Castle (Castelo de São Jorge).**
    Perched atop Lisbon's highest hill, St. George's Castle offers panoramic views and a deep dive into the city's ancient roots. This former Moorish fortification, later a royal palace, stands as a testament to centuries of conquests and cultural shifts. You can explore its battlements, towers, and archaeological site, and enjoy the impressive panorama of Lisbon. A typical visit lasts about 1 to 1.5 hours. The castle is a significant landmark, having been occupied successively by Phoenicians, Carthaginians, Romans, and Moors before its capture by the Portuguese in 1147.

*   **Evening (7:00 PM onwards): Dinner with Fado Music.**
    For dinner, consider heading back to Alfama, or the Bairro Alto neighborhood, both known for traditional Fado music performances. Fado, a soulful music tradition, is said to have been born in Alfama.

How does this sound for your first day? We can adjust anything you like!

--------------------------------------------------


Created a BRAND NEW session for Turn 2: 6599d10d-74e8-404c-8273-8c57d1305fec
🗣️ User (Turn 2): 'Yes, that looks perfect! Please plan Day 2.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '6599d10d-74e8-404c-8273-8c57d1305fec'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Here is your itinerary for Day 2 in Prague, focusing on a majestic castle, panoramic views, local flavors, and a touch of evening culture:

### **Day 2: Castle Grandeur, Panoramic Views, and Evening Jazz**

**Morning (9:00 AM - 1:00 PM): Discover the Grandeur of Prague Castle**
Start your day by exploring the magnificent **Prague Castle complex**, a UNESCO World Heritage site and the largest ancient castle complex in the world. This sprawling area is a showcase of various architectural styles and rich history. Allocate ample time to visit its key attractions:
*   **St. Vitus Cathedral:** Marvel at this Gothic masterpiece, known for its

Here is your itinerary for Day 2 in Prague, focusing on a majestic castle, panoramic views, local flavors, and a touch of evening culture:

### **Day 2: Castle Grandeur, Panoramic Views, and Evening Jazz**

**Morning (9:00 AM - 1:00 PM): Discover the Grandeur of Prague Castle**
Start your day by exploring the magnificent **Prague Castle complex**, a UNESCO World Heritage site and the largest ancient castle complex in the world. This sprawling area is a showcase of various architectural styles and rich history. Allocate ample time to visit its key attractions:
*   **St. Vitus Cathedral:** Marvel at this Gothic masterpiece, known for its intricate stained-glass windows, and the tombs of Czech kings and queens.
*   **Old Royal Palace:** Step into the historic Vladislav Hall, famous for its impressive ribbed vaulting.
*   **St. George's Basilica:** Explore this Romanesque basilica, one of the oldest churches within the castle complex.
*   **Golden Lane:** Wander through this charming, narrow street lined with colorful 16th-century houses, once home to castle guards and goldsmiths, now housing small shops and historical exhibits.

**Lunch (1:00 PM - 2:00 PM): Traditional Czech Lunch near Prague Castle**
After exploring the castle, descend into the Lesser Town (Malá Strana) for a traditional Czech lunch. Consider **Kuchyň** for authentic Czech dishes with views near the castle gates, or **Malostranská Beseda** for a local experience in a beautifully restored Renaissance building.

**Afternoon (2:00 PM - 5:30 PM): Ascend Petřín Hill for Panoramic Views and Greenery**
Head to **Petřín Hill**, offering a refreshing change of scenery and breathtaking panoramic views of Prague.
*   **Funicular Ride:** Take the scenic funicular railway up the hill from the Újezd tram stop (note: the funicular is closed for renovation until summer 2026, so consider walking or an alternative public transport route to reach the top).
*   **Petřín Lookout Tower:** Climb this mini Eiffel Tower for unparalleled 360-degree views across the entire city, including Prague Castle and the Vltava River. An elevator is available for those who prefer not to climb the 299 steps.
*   **Mirror Maze:** Enjoy a fun and whimsical experience in the Mirror Maze, which features both a labyrinth of mirrors and a "hall of laughter" with distorted reflections.

**Late Afternoon/Early Evening (5:30 PM - 7:00 PM): Relax and Stroll**
Descend Petřín Hill, perhaps taking a leisurely walk through the park's rose gardens or towards Kampa Island, allowing for some relaxation before dinner.

**Dinner (7:00 PM onwards): Authentic Czech Dinner**
For dinner, continue your exploration of local cuisine. **U Malého Glena** in Lesser Town is a popular spot that combines a bar with good food and a jazz club downstairs.

**Evening: Jazz Club Experience**
Immerse yourself in Prague's vibrant jazz scene. Prague has a strong jazz tradition and several excellent clubs. Options like **Reduta Jazz Club** or **AghaRTA Jazz Club** are well-known for their live performances and intimate atmosphere. Enjoy the music and a drink, soaking in the city's unique nightlife.

---

How does this plan for Day 2 sound? Would you like any adjustments, or are you ready to plan Day 3?

--------------------------------------------------



See? The agent was confused! It likely asked what destination or what trip we were talking about. Because the second query was in a fresh, isolated session, the agent had no memory of planning Day 1 in Lisbon.

This perfectly illustrates why **managing sessions is the key to building truly conversational agents!**

---
### Part 2: Multi-Agent Mayhem with `SequentialAgent` 🧠→⛓️→🤖

You've seen how to manually link agents together with custom Python code. It works, but it can get complicated. Now, let's refactor our workflow to use a powerful, built-in ADK feature designed specifically for this: the **`SequentialAgent`**.

The `SequentialAgent` is a *workflow agent*. It's not powered by an LLM itself; instead, its only job is to execute a list of other agents in a strict, predefined order.

The real magic ✨ is how it passes information. The ADK uses a shared `state` dictionary that each agent in the sequence can read from and write to.

**Our New Workflow:**

1.  **Foodie Agent**: Finds the restaurant and saves the name to `state['destination']`.
2.  **Transportation Agent**: Automatically reads `state['destination']` and uses it to find directions.

This means we no longer need custom Python code to extract text or build new queries! The ADK handles the plumbing for us.

```
+-------------------------------+
|  find_and_navigate_agent 🧭   |
| SequentialAgent:             |
| 1. Find destination          |
| 2. Get directions            |
+---------------+---------------+
                |
     +----------+----------+
     |                     |
     v                     v
+----------------+   +------------------------+
| foodie_agent 🍣 |   | transportation_agent 🚗 |
| Finds place     |   | Uses {destination}     |
| Output: 'Jin Sho'|   | Output: Directions     |
+----------------+   +------------------------+

Final Output: 🍣 Restaurant + 🚗 Route
```

Step 1: Define the Specialist "Worker" Agents
First, we define our individual specialist agents. The magic lies in two key parameters: output_key and the {placeholder} syntax.

The Information Producer (foodie_agent): This agent's job is to find a piece of information. We'll add an output_key to tell the framework where to save its final answer.

The Information Consumer (transportation_agent): This agent's job is to use the information from the first agent. We'll use a {placeholder} in its instructions that matches the output_key from the producer.



In [ ]:
# --- Agent Definitions for our Specialist Team (Refactored for Sequential Workflow) ---

# Note the new `output_key` and the more specific instruction.
foodie_agent = Agent(
    name="foodie_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are an expert food critic. Your goal is to find the best restaurant based on a user's request.

    When you recommend a place, you must output *only* the name of the establishment and nothing else.
    For example, if the best sushi is at 'Jin Sho', you should output only: Jin Sho
    """,
    output_key="destination"  # ADK will save the agent's final response to state['destination']
)

# The `{destination}` placeholder is automatically filled by the ADK from the state.
transportation_agent = Agent(
    name="transportation_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are a navigation assistant. Given a destination, provide clear directions.
    The user wants to go to: {destination}.

    Analyze the user's full original query to find their starting point.
    Then, provide clear directions from that starting point to {destination}.
    """,
)

# This agent will run foodie_agent, then transportation_agent, in that exact order.
find_and_navigate_agent = SequentialAgent(
    name="find_and_navigate_agent",
    sub_agents=[foodie_agent, transportation_agent],
    description="A workflow that first finds a location and then provides directions to it."
)

In [ ]:
# --- Let's Test the Streamlined Workflow! ---

async def run_sequential_workflow():
    """
    A simplified test function that directly invokes the SequentialAgent.
    """

    # The query contains all the information needed for the entire sequence.
    query = "Find me the best sushi restaurant in Palo Alto, and then tell me how to get there from the downtown Caltrain station."

    print(f"\n{'='*60}\n🗣️  Processing Query: '{query}'\n{'='*60}")
    print(f"🚀 Handing off the entire task to the '{find_and_navigate_agent.name}'...")

    # 1. Create a single session for our sequential agent
    # The session will manage the state (like the 'destination' variable) across the sub-agent calls.
    session = await session_service.create_session(app_name=find_and_navigate_agent.name, user_id=my_user_id)

    # 2. Run the query
    # The SequentialAgent will automatically:
    #   - Call foodie_agent with the query.
    #   - Take its output and save it to the state as `state['destination']`.
    #   - Call transportation_agent, injecting the destination into its prompt.
    #   - Stream the final response from the transportation_agent.
    await run_agent_query(find_and_navigate_agent, query, session, my_user_id)

    print(f"\n--- ✅ '{find_and_navigate_agent.name}' Workflow Complete ---")


# Execute the simplified test
await run_sequential_workflow()

---
## 🎉 Congratulations! 🎉

You've completed the Enhanced ADK Adventure! You have successfully:

1.  **Built a constrained agent** that understands budget and context.
2.  **Created an adaptive agent** that can handle feedback and re-plan on the fly.
3.  **Orchestrated a sequential agent workflow**, making a team of agents collaborate to solve a complex task.

You now have the fundamental building blocks to create incredibly sophisticated AI applications. The only limit is your imagination.

**What will you build next?**

```
   __            /\_/\         /\_/\        /\_/\         __             (\__/)
o-''|\_____/).  ( o.o )       ( -.- )      ( ^_^ )     o-''|\_____/).    ( ^_^ )
 \_/|_)     )    > ^ <         > * <        >💖<         \_/|_)     )     / >🌸< \
    \  __  /                                              \  __  /         /   \
    (_/ (_/                                               (_/ (_/        (___|___)
```
